In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
df=pd.read_csv("../data/visualized_water_quality.csv", parse_dates=['Sample Date'])

In [3]:
df.head()

,Sample Site,Sample Date,Analyte,Final Result,Units,Stream Group,Year,Log Final Result,Month,Season
0,S4,1987-01-07,Temperature,1.111110,C,Schoharie,1987,0.105360,1,Winter
1,S6I,1987-01-07,Temperature,0.555556,C,Schoharie,1987,-0.587786,1,Winter
2,S8,1987-01-07,Temperature,2.222220,C,Schoharie,1987,0.798507,1,Winter
3,S4,1987-01-07,Scent Character,6.970000,mg/L,Schoharie,1987,1.941615,1,Winter
4,S6I,1987-01-07,Scent Character,6.970000,mg/L,Schoharie,1987,1.941615,1,Winter


In [4]:
# label encoding for categorical variables i.e. Stream Group encoded and Season encoded
le_stream_group=LabelEncoder()
le_season=LabelEncoder()

df['Stream Group Encoded']=le_stream_group.fit_transform(df['Stream Group'])
df['Season Encoded']=le_season.fit_transform(df['Season'])

In [5]:
#encoding sample site for location based analysis
le_sample_site=LabelEncoder()
df['Sample Site Encoded']=le_sample_site.fit_transform(df['Sample Site'])

In [6]:
# scaling numerical features i.e final result and log final result
scaler=StandardScaler()
df['Final Result Scaled']=scaler.fit_transform(df[['Final Result']])
df['Log Final Result Scaled']=scaler.fit_transform(df[['Log Final Result']])

In [7]:
#creating new features
df['Day']=df['Sample Date'].dt.day
df['Weekday']=df['Sample Date'].dt.weekday

In [11]:
#creating rolling averages for key analytes like turbidity (rolling average for 7 days)

#filtering turbidity data and ensuring a deep copy
turbidity_data=df[df['Analyte']=='Turbidity'].copy()

#aplying rolling average calculation safely
turbidity_data.loc[:,'Turbidity Rolling Avg']=turbidity_data['Final Result'].rolling(window=7, min_periods=1).mean()

#verifying the new column
print(turbidity_data[['Sample Date', 'Final Result', 'Turbidity Rolling Avg']].head())

    Sample Date  Final Result  Turbidity Rolling Avg
32   1987-01-12           3.0                   3.00
183  1987-01-20           0.8                   1.90
184  1987-01-20           0.4                   1.40
185  1987-01-20           0.4                   1.15
186  1987-01-20           0.6                   1.04


In [13]:
#merging back into the main dataframe
df=df.merge(turbidity_data[['Sample Site', 'Sample Date', 'Turbidity Rolling Avg']], on=['Sample Site', 'Sample Date'],how='left')

In [14]:
df.head()

,Sample Site,Sample Date,Analyte,Final Result,Units,Stream Group,Year,Log Final Result,Month,Season,Stream Group Encoded,Season Encoded,Sample Site Encoded,Final Result Scaled,Log Final Result Scaled,Day,Weekday,Turbidity Rolling Avg
0,S4,1987-01-07,Temperature,1.111110,C,Schoharie,1987,0.105360,1,Winter,14,3,340,-0.035130,-0.750655,7,2,1.314286
1,S6I,1987-01-07,Temperature,0.555556,C,Schoharie,1987,-0.587786,1,Winter,14,3,343,-0.035349,-1.074265,7,2,1.257143
2,S8,1987-01-07,Temperature,2.222220,C,Schoharie,1987,0.798507,1,Winter,14,3,355,-0.034693,-0.427043,7,2,1.471429
3,S4,1987-01-07,Scent Character,6.970000,mg/L,Schoharie,1987,1.941615,1,Winter,14,3,340,-0.032826,0.106643,7,2,1.314286
4,S6I,1987-01-07,Scent Character,6.970000,mg/L,Schoharie,1987,1.941615,1,Winter,14,3,343,-0.032826,0.106643,7,2,1.257143
